# Tutorial: Librería de Difusión y Descomposición de Redes

Este cuaderno explica el funcionamiento de la librería personalizada `difusion_lib`. Esta herramienta está diseñada para analizar la estructura jerárquica de grafos dirigidos mediante la simulación de flujo de información (difusión) y la eliminación iterativa de nodos dominantes (pelado).

## 1. Arquitectura de la Librería

La librería se divide en cuatro módulos principales que trabajan en conjunto bajo el patrón **Modelo-Vista-Controlador (MVC)**:

### A. `motor_difusion.py` (El Motor Físico)
Es el núcleo matemático. En lugar de iterar nodo por nodo (que es lento), convierte el grafo en una **Matriz de Transición Dispersa ($M$)**.
- **Física:** Calcula $v_{t+1} = M \cdot v_t$.
- **Optimización:** Utiliza `scipy.sparse` para manejar grandes grafos eficientemente.
- **Normalización:** Los pesos de salida se normalizan por el grado de salida del nodo, aplicándole una `tasa_difusion` (factor de amortiguación).

### B. `controlador.py` (El Cerebro)
Orquesta la simulación. Tiene dos modos principales:
1. **`ejecutar_estudio`**: Simulación de difusión estándar. Útil para ver hasta dónde llega la información desde un punto.
2. **`ejecutar_estudio_pelado`**: El algoritmo principal. 
   - Ejecuta difusión -> Identifica nodos con masa > umbral -> Los elimina -> Repite.
   - Esto permite separar las capas "Globales" (influencers) de las capas "Locales" (comunidades).

### C. `visualizador.py` (La Vista)
Genera reportes visuales avanzados.
- **3D Interactivo:** Usa `plotly` para crear archivos HTML donde se puede rotar el grafo y ver las flechas de flujo.
- **Mega Dashboard:** Agrupa múltiples simulaciones y capas de pelado en un solo HTML con menús desplegables.
- **Estático:** Genera imágenes PNG con `matplotlib`.

### D. `generaradores.py` (El Entorno)
Contiene métodos estáticos para crear topologías de red sintéticas:
- `malla_netlogo`: Una rejilla con probabilidad de conexión estocástica.
- `red_social_realista`: Modelo Holme-Kim modificado con lógica de "Lurkers" (usuarios pasivos) y direccionalidad estricta.
- `gaussiana`, `sbm`, `cascada`, etc.

## 2. Configuración del Entorno

Asegúrate de que la carpeta `difusion_lib` esté en el mismo directorio que este notebook.

In [5]:
import os
import pandas as pd
import networkx as nx
from datetime import datetime

# Importamos los módulos de tu librería
from difusion_lib import ControladorPelado, VisualizadorPelado, GeneradorRedes

## 3. El Script de Batería Masiva

El siguiente script es una función envolvente ("wrapper") diseñada para automatizar experimentos científicos. 

### ¿Qué hace este script?
1. **Generación:** Crea $N$ grafos diferentes basados en los parámetros aleatorios.
2. **Pelado (Decomposition):** Desmonta el grafo capa por capa eliminando los nodos con más "masa" (influencia).
3. **Prueba de Resiliencia (Difusión Final):** Toma los nodos que sobrevivieron al pelado (el "Core") e inyecta masa en ellos sobre el grafo original restaurado. Esto mide si el núcleo restante es capaz de influir en toda la red o si está aislado.
4. **Reporte:** Consolida métricas en CSV y genera un Dashboard 3D unificado.

### Uso de `**kwargs`
Observa el uso de `**kwargs_grafo`. Esto permite pasar parámetros específicos (como `dim` para mallas o `p_triangle` para redes sociales) sin tener que reescribir la función principal.

In [6]:
def cantidad_nodos_mojados(record):
    """Cuenta cuántos nodos tienen un valor > 0 en el registro de difusión."""
    return sum(1 for i in record if i > 0)

def ejecutar_bateria_masiva(
    tipo_de_grafo='malla_netlogo', 
    n_simulaciones=5, 
    master_folder="simulaciones", 
    tasa_difusion=0.2, 
    num_pelados=10, 
    iteraciones_por_pelado=150, 
    iteraciones_difusion=150, 
    umbral_masa=1.0, 
    umbral_nodos_final=1, 
    masa_total_concentrada=100,
    **kwargs_grafo # Argumentos dinámicos para el generador
):
    # Diccionario 'Dispatcher' para seleccionar el algoritmo de generación
    mapeo_generadores = {
        'malla_netlogo': GeneradorRedes.generar_malla_estocastica_netlogo,
        'cascada': GeneradorRedes.generar_cascada_estricta,
        'flujo_libre': GeneradorRedes.generar_flujo_libre_escala,
        'sbm': GeneradorRedes.generar_sbm_estocastico,
        'gaussiana': GeneradorRedes.generar_red_gaussiana,
        'red_social_realista' : GeneradorRedes.generar_red_social_realista
    }

    if tipo_de_grafo not in mapeo_generadores:
        raise ValueError(f"Grafo '{tipo_de_grafo}' no reconocido. Opciones: {list(mapeo_generadores.keys())}")

    if not os.path.exists(master_folder):
        os.makedirs(master_folder)
        print(f"Carpeta maestra creada: {master_folder}")

    mega_recolector_figs = {} 
    resumen_metricas = []     

    print(f"=== INICIANDO BATERÍA: {tipo_de_grafo.upper()} (N={n_simulaciones}) ===")
    print(f"Parámetros del grafo: {kwargs_grafo}")

    for i in range(n_simulaciones):
        sim_id = f"Simulacion_{i+1:03d}" 
        print(f"\n>>> Procesando: {sim_id}")

        # 1. GENERACIÓN DEL GRAFO
        func_generadora = mapeo_generadores[tipo_de_grafo]
        resultado_generador = func_generadora(**kwargs_grafo)

        # Manejo de grafos que devuelven tuplas (como la gaussiana)
        if isinstance(resultado_generador, tuple):
            G_original = resultado_generador[0]
        else:
            G_original = resultado_generador

        n_total_nodos = len(G_original)

        # 2. PROCESO DE PELADO (Descomposición)
        ctrl_peel = ControladorPelado(G_original)
        folder_sim = os.path.join(master_folder, sim_id)
        
        _, figs_peel, G_survivors, pelados_dict = ctrl_peel.ejecutar_estudio_pelado(
            num_pelados=num_pelados,                  
            iteraciones_por_pelado=iteraciones_por_pelado,
            umbral_masa=umbral_masa,
            umbral_nodos_final=umbral_nodos_final, 
            tasa_difusion=tasa_difusion,
            exportar_resultados=True,
            carpeta_exportacion=folder_sim
        )

        num_peels = len(pelados_dict)
        if num_peels > 0:
            last_layer_idx = max(pelados_dict.keys())
            nodos_ultima_capa_eliminada = len(pelados_dict[last_layer_idx])
        else:
            nodos_ultima_capa_eliminada = 0
            
        nodos_sobrevivientes = len(G_survivors)

        # 3. DIFUSIÓN FINAL (Prueba de Core)
        ctrl_diff = ControladorPelado(G_original)
        folder_diff = os.path.join(folder_sim, "Difusion_Core")
        
        _, figs_diff, record_final = ctrl_diff.ejecutar_estudio(
            iteraciones=iteraciones_difusion,
            nodos=list(G_survivors.nodes()), 
            tasa_difusion=tasa_difusion,
            valor_inicio=masa_total_concentrada,             
            exportar_resultados=True,
            carpeta_exportacion=folder_diff,
            nombre_resumen="resumen_difusion_final.csv"
        )
        
        n_mojados = cantidad_nodos_mojados(record_final)
        
        ratio_val = float(n_mojados) / float(n_total_nodos) if n_total_nodos > 0 else 0.0

        # Recolección para Dashboard
        mega_recolector_figs[f"{sim_id} - Pelado"] = figs_peel
        mega_recolector_figs[f"{sim_id} - Difusion"] = figs_diff

        # Métricas
        resumen_metricas.append({
            "Simulacion_ID": sim_id,
            "Tipo_Grafo": tipo_de_grafo, 
            "Total_Nodos_Inicial": n_total_nodos,
            "Total_Capas_Peladas": num_peels,
            "Nodos_Ultima_Capa_Pelada": nodos_ultima_capa_eliminada,
            "Nodos_Sobrevivientes": nodos_sobrevivientes,
            "Nodos_Mojados": n_mojados,
            "Ratio_Mojados": ratio_val
        })

        print(f"   -> Pelados: {num_peels} | Mojados: {n_mojados} | Ratio: {ratio_val:.4f}")

    print("\nGenerando archivos maestros...")

    # CSV Consolidado
    df_maestro = pd.DataFrame(resumen_metricas)
    path_csv = os.path.join(master_folder, "Metricas_Consolidadas.csv")
    df_maestro.to_csv(path_csv, index=False)
    
    # CSV Promedios
    cols_numericas = [
        "Total_Nodos_Inicial", "Total_Capas_Peladas", 
        "Nodos_Ultima_Capa_Pelada", "Nodos_Sobrevivientes", 
        "Nodos_Mojados", "Ratio_Mojados"
    ]
    for col in cols_numericas: df_maestro[col] = pd.to_numeric(df_maestro[col])
    
    df_promedios = df_maestro[cols_numericas].mean().to_frame(name="Promedio").T
    path_avg = os.path.join(master_folder, "Promedios_Consolidados.csv")
    df_promedios.to_csv(path_avg, index=False)

    # HTML Dashboard 3D
    VisualizadorPelado.exportar_mega_dashboard(
        mega_recolector_figs,
        master_folder,
        "Dashboard_Global_3D.html"
    )
    print("=== BATERÍA COMPLETADA ===")

## 4. Ejecución del Experimento

A continuación, ejecutaremos dos experimentos distintos para ver cómo la topología afecta la difusión.

1. **Malla NetLogo:** Simula un terreno físico o sensores conectados por proximidad.
2. **Red Social Realista:** Simula Twitter/Instagram, con "Lurkers" y conexiones direccionales.

In [7]:
marca_tiempo = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
dim_malla = 5

# EXPERIMENTO 1: Malla Estocástica (NetLogo Style)
# ------------------------------------------------
print("--- EJECUTANDO ESTUDIO MALLA NETLOGO ---")
ejecutar_bateria_masiva(
    tipo_de_grafo='malla_netlogo', 
    n_simulaciones=5, 
    master_folder=f"simulaciones/Estudio_{marca_tiempo}/Estudio_NETLOGO", 
    # Parámetros de simulación
    tasa_difusion=0.2,       # Amortiguación (más bajo = la señal muere antes)
    num_pelados=10,          # Máximo capas a quitar
    iteraciones_por_pelado=150, 
    umbral_masa=1.0,         # Cuánta masa necesita un nodo para ser eliminado
    # Parámetros del Generador (kwargs)
    dim=dim_malla,           
    link_chance=60           # Probabilidad de conexión (60%)
)

--- EJECUTANDO ESTUDIO MALLA NETLOGO ---
Carpeta maestra creada: simulaciones/Estudio_2026-01-18_01-48-25/Estudio_NETLOGO
=== INICIANDO BATERÍA: MALLA_NETLOGO (N=5) ===
Parámetros del grafo: {'dim': 5, 'link_chance': 60}

>>> Procesando: Simulacion_001
Iniciando Estudio: 25 nodos.
Pelado 1: Eliminando 7 componentes.
Pelado 2: Eliminando 4 componentes.
Pelado 3: Eliminando 4 componentes.
Pelado 4: Eliminando 5 componentes.
Pelado 5: Eliminando 5 componentes.
Iniciando Difusión: 25 nodos.
   -> Pelados: 4 | Mojados: 21 | Ratio: 0.8400

>>> Procesando: Simulacion_002
Iniciando Estudio: 25 nodos.
Pelado 1: Eliminando 7 componentes.
Pelado 2: Eliminando 8 componentes.
Pelado 3: Eliminando 4 componentes.
Pelado 4: Eliminando 5 componentes.
Pelado 5: Eliminando 1 componentes.
Iniciando Difusión: 25 nodos.
   -> Pelados: 4 | Mojados: 24 | Ratio: 0.9600

>>> Procesando: Simulacion_003
Iniciando Estudio: 25 nodos.
Pelado 1: Eliminando 5 componentes.
Pelado 2: Eliminando 6 componentes.
Pelado 3: 

In [ ]:
# EXPERIMENTO 2: Red Social Realista
# ----------------------------------
print("\n--- EJECUTANDO ESTUDIO RED SOCIAL ---")
ejecutar_bateria_masiva(
    tipo_de_grafo='red_social_realista', 
    n_simulaciones=5, 
    master_folder=f"simulaciones/Estudio_GUIA{marca_tiempo}/red_social_realista", 
    tasa_difusion=0.3,       # Un poco más alto para redes sociales
    num_pelados=15, 
    # Parámetros del Generador (kwargs)
    n_users=300,             # Usuarios totales
    m_neighbors=2,           # Vecinos iniciales (red dispersa)
    p_triangle=0.3,          # Probabilidad de clúster (amigos de amigos)
    ratio_mutual=0.05        # Solo 5% de conexiones mutuas (tipo Twitter)
)


--- EJECUTANDO ESTUDIO RED SOCIAL ---
Carpeta maestra creada: simulaciones/Estudio_2026-01-18_01-48-25/red_social_realista
=== INICIANDO BATERÍA: RED_SOCIAL_REALISTA (N=5) ===
Parámetros del grafo: {'n_users': 300, 'm_neighbors': 2, 'p_triangle': 0.3, 'ratio_mutual': 0.05}

>>> Procesando: Simulacion_001
Iniciando Estudio: 300 nodos.
Pelado 1: Eliminando 101 componentes.
Pelado 2: Eliminando 46 componentes.
Pelado 3: Eliminando 14 componentes.
Pelado 4: Eliminando 25 componentes.
Pelado 5: Eliminando 20 componentes.
Pelado 6: Eliminando 8 componentes.
Pelado 7: Eliminando 6 componentes.
Pelado 8: Eliminando 10 componentes.
Pelado 9: Eliminando 26 componentes.
Pelado 10: Eliminando 8 componentes.
Pelado 11: Eliminando 6 componentes.
Pelado 12: Eliminando 9 componentes.
Pelado 13: Eliminando 9 componentes.
Pelado 14: Eliminando 9 componentes.
Pelado 15: Eliminando 3 componentes.
Iniciando Difusión: 300 nodos.


## 5. Interpretación de Resultados

Una vez finalizada la ejecución, ve a la carpeta `simulaciones/Estudio_[fecha]`. Encontrarás:

1. **`Dashboard_Global_3D.html`**: Abre este archivo en tu navegador.
   - Usa el menú desplegable arriba a la izquierda para cambiar entre simulaciones.
   - Observa cómo las flechas indican la dirección del flujo.
   - Los nodos más grandes y rojos tienen más "masa".

2. **`Promedios_Consolidados.csv`**: Revisa la columna `Ratio_Mojados`.
   - Si es **1.0**, la red está totalmente conectada (típico en mallas densas).
   - Si es **0.4 - 0.7**, la red tiene comunidades aisladas o "Sinks" (típico en la red social realista).